In [271]:
#!conda install -c anaconda beautifulsoup4

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from pandas.io.html import read_html
import numpy as np

## Scrape Wikipedia Page for Table containing Postcode, Borough, Neighbourhood

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
#soup = BeautifulSoup(page, "lxml")

### Extract table from wikipedia page into dataframe

In [3]:
#print(soup.prettify())
#table_wiki = soup.find("table", class_="wikitable sortable")
df = read_html(url, attrs={"class":"wikitable sortable"})[0]
print("Shape: ", df.shape)
df.head()


Shape:  (288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Preprocessing Data

### Drop rows with borough = 'Not assigned'

In [4]:
# check for boroughs with values "Not assigned"
df_NotAssigned = df[df["Borough"]=="Not assigned"]
print("Shape: ", df_NotAssigned.shape)
df_NotAssigned.head()

Shape:  (77, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
9,M8A,Not assigned,Not assigned
13,M2B,Not assigned,Not assigned
20,M7B,Not assigned,Not assigned


In [5]:
df.drop(df[df.Borough=="Not assigned"].index, axis=0, inplace=True)
print("Shape: ", df.shape)
df.head()

Shape:  (211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Replace Neighbourhoods containing  'Not assigned' with Borough name

In [6]:
# Check for neighbourhoods with 'Not assigned' values
df_NotAssigned = df[df["Neighbourhood"]=="Not assigned"]
print("Shape: ", df_NotAssigned.shape)
df_NotAssigned.head()

Shape:  (1, 3)


,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [7]:
# Replace neighbourhoods with 'Not assigned' to borough
i = (df.Neighbourhood=="Not assigned")
df.loc[i, "Neighbourhood"] = df.loc[i, "Borough"]
print("Shape: ",df.shape)
df.head(8)

Shape:  (211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue


### Combine neighbourhoods with the same postal codes into a comma separated list

In [29]:
df=df.groupby(["Postcode","Borough"])["Neighbourhood"].agg([("Neighbourhood", ", ".join)]).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
df.shape

(103, 3)

## Obtain Geographical Coordinates for each PostCode

In [38]:
#Obtain file with geographical coordinates
!wget -O geo_data.csv https://cocl.us/Geospatial_data

--2019-09-29 11:41:31--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-09-29 11:41:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-09-29 11:41:34--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [39]:
df_geo = pd.read_csv("geo_data.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Create 1 combined table with Postcode info and Geographical Coodinates

In [44]:
df2 = pd.merge(left=df, right=df_geo, how="left", left_on="Postcode", right_on="Postal Code")
print("Shape: ",df2.shape)
df2.head()

Shape:  (103, 6)


,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Explore and Cluster Neighbourhoods in Toronto

### Create a Dataframe containing Toronto Neighbourhoods

In [140]:
df_t = df2[df2["Borough"].str.contains("Toronto")].reset_index()
df_t.drop(labels=["index","Postal Code"],axis=1,inplace=True)
print("Shape: ", df_t.shape)
df_t.head()

Shape:  (38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Import Libraries

In [61]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [56]:
# The code was removed by Watson Studio for sharing.

### Get nearby venues from foursquare

In [194]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [195]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
venues = getNearbyVenues(names=df_t['Neighbourhood'],latitudes=df_t['Latitude'],longitudes=df_t['Longitude'])


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [196]:
print("Shape: ", venues.shape)
venues.head()

Shape:  (1704, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [197]:
venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",49,49,49,49,49,49
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


### Analyse each neighbourhood

In [259]:
venues_pop = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_pop.drop("Neighborhood",axis=1,inplace=True)
venues_pop["Neighborhood"] = venues["Neighborhood"] 

# move neighborhood column to the first column
fixed_columns = ["Neighborhood"] + list(venues_pop.columns[:-1])
venues_pop = venues_pop[fixed_columns]

print("Shape: ", venues_pop.shape)
venues_pop.head()

Shape:  (1704, 233)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [260]:
venues_grouped = venues_pop.groupby("Neighborhood").sum().reset_index()
venues_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,3,0,...,0,0,0,0,2,0,0,1,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,"Brockton, Exhibition Place, Parkdale Village",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0,1,1,1,2,3,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Cabbagetown, St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Central Bay Street,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
7,"Chinatown, Grange Park, Kensington Market",0,0,0,0,0,0,0,0,0,...,0,1,0,0,6,0,4,1,0,0
8,Christie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Church and Wellesley,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,1,0,1,1


In [261]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [262]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Hotel,Cosmetics Shop,Restaurant,Thai Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Café,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Gym,Burrito Place,Caribbean Restaurant,Stadium,Restaurant,Climbing Gym,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Skate Park,Comic Shop,Farmers Market,Fast Food Restaurant,Burrito Place,Smoke Shop,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Sculpture Garden,Boat or Ferry,Boutique,Airport Food Court


### Cluster Neighbourhoods and Display Clusters

In [263]:
# set number of clusters
kclusters = 5
venues_grouped_clustering = venues_grouped.drop('Neighborhood', 1)
venues_grouped_clustering["Total"] = venues_grouped_clustering.sum(axis=1,skipna=True)
venues_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)


In [264]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

df_t_merged = df_t[:]
df_t_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_t_merged = pd.merge(left=df_t_merged, right=neighborhoods_venues_sorted, how="left", left_on="Neighbourhood", right_on="Neighborhood")
df_t_merged.drop("Neighborhood",axis=1,inplace=True)
df_t_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Pub,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Café
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Pub,Brewery,Burrito Place,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Cheese Shop,Bookstore,Fish Market,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [265]:
# create map
map_clusters = folium.Map(location=[43.728020, -79.388790], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_t_merged['Latitude'], df_t_merged['Longitude'], df_t_merged['Neighbourhood'], df_t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyse Each Neighbourhood

#### Cluster 0: More frequent activities involving shopping

In [266]:
df_t_merged.loc[df_t_merged['Cluster Labels'] == 0, df_t_merged.columns[[1] + list(range(5, df_t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,0,Park,Sandwich Place,Pub,Brewery,Burrito Place,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
6,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Furniture / Home Store,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant
9,Central Toronto,0,Pub,Coffee Shop,Pizza Place,Light Rail Station,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Liquor Store,Fried Chicken Joint
24,Central Toronto,0,Coffee Shop,Café,Sandwich Place,Pizza Place,Liquor Store,Park,Jewish Restaurant,Indian Restaurant,BBQ Joint,Pub
27,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Sculpture Garden,Boat or Ferry,Boutique,Airport Food Court
30,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Diner,Restaurant,Nightclub,Coffee Shop,Convenience Store,Italian Restaurant
31,West Toronto,0,Supermarket,Pharmacy,Bakery,Café,Bar,Bank,Brewery,Middle Eastern Restaurant,Music Venue,Art Gallery
33,West Toronto,0,Coffee Shop,Breakfast Spot,Café,Gym,Burrito Place,Caribbean Restaurant,Stadium,Restaurant,Climbing Gym,Bar
34,West Toronto,0,Mexican Restaurant,Café,Grocery Store,Fried Chicken Joint,Music Venue,Diner,Discount Store,Cajun / Creole Restaurant,Fast Food Restaurant,Bookstore
35,West Toronto,0,Gift Shop,Breakfast Spot,Bookstore,Dessert Shop,Eastern European Restaurant,Bar,Bank,Movie Theater,Dog Run,Restaurant


### Cluster 1: More frequent activities at cafes and hotels

In [267]:
df_t_merged.loc[df_t_merged['Cluster Labels'] == 1, df_t_merged.columns[[1] + list(range(5, df_t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Men's Store,Hotel,Fast Food Restaurant,Mediterranean Restaurant,Gastropub
14,Downtown Toronto,1,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Cosmetics Shop,Café,Ramen Restaurant,Restaurant,Bookstore,Italian Restaurant,Japanese Restaurant
15,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Breakfast Spot,Clothing Store,Cosmetics Shop,Beer Bar,Bakery
17,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Ice Cream Shop,Sushi Restaurant,Japanese Restaurant,Spa
18,Downtown Toronto,1,Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Hotel,Cosmetics Shop,Restaurant,Thai Restaurant,American Restaurant
19,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Italian Restaurant,Restaurant,Baseball Stadium
20,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Gastropub,Bar,Deli / Bodega,Bakery,American Restaurant
21,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Steakhouse,Gym,Deli / Bodega
26,Downtown Toronto,1,Café,Vegetarian / Vegan Restaurant,Bar,Bakery,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Park
28,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Beer Bar,Hotel,Japanese Restaurant,Breakfast Spot


### Cluster 2: More frequent activities in places associated with cafes, deserts and restaurants

In [268]:
df_t_merged.loc[df_t_merged['Cluster Labels'] == 2, df_t_merged.columns[[1] + list(range(5, df_t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Café
3,East Toronto,2,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Cheese Shop,Bookstore,Fish Market,Brewery,Seafood Restaurant
7,Central Toronto,2,Sandwich Place,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Coffee Shop,Pizza Place,Greek Restaurant,Seafood Restaurant
25,Downtown Toronto,2,Café,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Restaurant,Bakery,Chinese Restaurant,Beer Bar,Beer Store
36,West Toronto,2,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Electronics Store,Food & Drink Shop,Food,Fish & Chips Shop,Bookstore


### Cluster 3: More frequent activities in places with coffee and alcohol

In [269]:
df_t_merged.loc[df_t_merged['Cluster Labels'] == 3, df_t_merged.columns[[1] + list(range(5, df_t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,3,Coffee Shop,Park,Italian Restaurant,Café,Restaurant,Market,Pub,Bakery,Pizza Place,General Entertainment
13,Downtown Toronto,3,Coffee Shop,Pub,Bakery,Park,Café,Mexican Restaurant,Restaurant,Gym / Fitness Center,Breakfast Spot,Theater
16,Downtown Toronto,3,Coffee Shop,Cocktail Bar,Italian Restaurant,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Café,Steakhouse
32,West Toronto,3,Bar,Asian Restaurant,Coffee Shop,Cocktail Bar,Boutique,Vietnamese Restaurant,Pizza Place,Men's Store,French Restaurant,Restaurant


### Cluster 4: More frequent activities in places related to exercise and healthy foods

In [270]:
df_t_merged.loc[df_t_merged['Cluster Labels'] == 4, df_t_merged.columns[[1] + list(range(5, df_t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,4,Health Food Store,Pub,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,Central Toronto,4,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,Central Toronto,4,Breakfast Spot,Park,Food & Drink Shop,Clothing Store,Hotel,Gym,Sandwich Place,Ethiopian Restaurant,Event Space,Electronics Store
8,Central Toronto,4,Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,Downtown Toronto,4,Park,Playground,Trail,Building,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
22,Central Toronto,4,Garden,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
23,Central Toronto,4,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
